<a href="https://colab.research.google.com/github/ullasbc-02/natural-language-understanding/blob/main/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1

In [50]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [4]:
df = pd.read_csv('Tweets_5K.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   textID     5000 non-null   object
 1   text       5000 non-null   object
 2   sentiment  5000 non-null   object
dtypes: object(3)
memory usage: 117.3+ KB


In [5]:
raw_tweets = df['text']
sentiments = df['sentiment']

In [6]:
senti_mapping = {
    'negative': -1,
    'neutral': 0,
    'positive': 1
}

labels = df['sentiment'].map(senti_mapping).tolist()

In [7]:
raw_tweets.head(10)

,text
0,"I`d have responded, if I were going"
1,Sooo SAD I will miss you here in San Diego!!!
2,my boss is bullying me...
3,what interview! leave me alone
4,"Sons of ****, why couldn`t they put them on t..."
5,http://www.dothebouncy.com/smf - some shameles...
6,2am feedings for the baby are fun when he is a...
7,Soooo high
8,Both of you
9,Journey!? Wow... u just became cooler. hehe....


In [8]:
print(labels)

[0, -1, -1, -1, -1, 0, 1, 0, 0, 1, 0, 1, -1, -1, 0, -1, -1, -1, -1, 0, 0, 1, 0, 0, 0, 1, -1, -1, 1, -1, 1, 1, -1, 1, 0, 0, -1, 0, -1, 1, 0, 1, 0, 0, 1, 0, -1, 0, -1, -1, 0, 0, 0, -1, 1, 0, -1, 0, -1, 0, -1, 0, -1, 1, -1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, -1, 0, -1, 0, 1, 1, 0, -1, -1, 1, 0, 0, -1, 1, 1, 0, 0, -1, 0, 1, -1, -1, 0, 1, 1, 0, -1, -1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, -1, -1, 1, 1, 0, 0, 0, -1, -1, -1, 1, 0, 0, 0, 1, 1, 1, -1, -1, 1, 1, 0, 0, 0, -1, 0, -1, -1, -1, -1, 0, -1, 0, 1, 1, 0, 1, -1, 1, -1, 0, 0, 0, -1, 1, -1, 0, 0, -1, 0, -1, 1, 0, 1, 1, 1, 1, -1, 0, 0, -1, -1, 0, -1, 0, 0, 1, 1, -1, 0, 0, 0, -1, 0, -1, 1, 1, 0, -1, -1, 0, -1, 1, 0, 0, -1, -1, 0, 0, -1, -1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, -1, 0, 0, 0, 0, -1, 0, -1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, -1, -1, 0, 0, 0, 1, 0, -1, 0, 0, 1, 0, 0, -1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, -1, -1, 1, -1, 1, 1, 0, -1, 0, -1, 0, -1, 0, 0, 1, -1, -1, 0, 0, 1, 0, 0, 0, 1, 1

In [9]:
basic_preproc_tweets = raw_tweets.str.strip().str.split(" ")

basic_preproc_tweets.head(10)

,text
0,"[I`d, have, responded,, if, I, were, going]"
1,"[Sooo, SAD, I, will, miss, you, here, in, San,..."
2,"[my, boss, is, bullying, me...]"
3,"[what, interview!, leave, me, alone]"
4,"[Sons, of, ****,, why, couldn`t, they, put, th..."
5,"[http://www.dothebouncy.com/smf, -, some, sham..."
6,"[2am, feedings, for, the, baby, are, fun, when..."
7,"[Soooo, high]"
8,"[Both, of, you]"
9,"[Journey!?, Wow..., u, just, became, cooler., ..."


In [10]:
basic_preproc_bow = basic_preproc_tweets.apply(lambda x: pd.Series(x).value_counts()).fillna(0)

In [11]:
basic_preproc_bow.shape

(5000, 15122)

In [12]:
print(basic_preproc_bow)

      I`d  have  responded,   if    I  were  going  Sooo  SAD  will  ...  \
0     1.0   1.0         1.0  1.0  1.0   1.0    1.0   0.0  0.0   0.0  ...   
1     0.0   0.0         0.0  0.0  1.0   0.0    0.0   1.0  1.0   1.0  ...   
2     0.0   0.0         0.0  0.0  0.0   0.0    0.0   0.0  0.0   0.0  ...   
3     0.0   0.0         0.0  0.0  0.0   0.0    0.0   0.0  0.0   0.0  ...   
4     0.0   0.0         0.0  0.0  0.0   0.0    0.0   0.0  0.0   0.0  ...   
...   ...   ...         ...  ...  ...   ...    ...   ...  ...   ...  ...   
4995  0.0   0.0         0.0  0.0  0.0   0.0    0.0   0.0  0.0   0.0  ...   
4996  0.0   0.0         0.0  0.0  0.0   0.0    0.0   0.0  0.0   0.0  ...   
4997  0.0   0.0         0.0  0.0  0.0   0.0    0.0   0.0  0.0   0.0  ...   
4998  0.0   0.0         0.0  0.0  1.0   0.0    0.0   0.0  0.0   0.0  ...   
4999  0.0   1.0         0.0  0.0  0.0   0.0    0.0   1.0  0.0   0.0  ...   

      reviews  scrapbook.  fiasco  wrapped  midterms   =(  7th  headache-all  \
0      

In [13]:
print(basic_preproc_bow.iloc[1460, 1460])
print(basic_preproc_bow.columns[1460])
#1460 document does not contain word "300th", so the value is 0 otherwise it will be 1.1
#1460the column signifies the word "300th"

0.0
300th


In [14]:
Tweets_train, Tweets_test, Labels_train, Labels_test = train_test_split(basic_preproc_bow, labels, test_size=0.2, shuffle=False)

# Part 2

In [25]:

model_nb = MultinomialNB(alpha=1.0)

model_nb.fit(Tweets_train, Labels_train)

nb_predictions = model_nb.predict(Tweets_test)
nb_accuracy = accuracy_score(Labels_test, nb_predictions)
print("Naive Bayes Accuracy:",nb_accuracy)

Naive Bayes Accuracy: 0.577


In [28]:

baseline_model = DummyClassifier(strategy="most_frequent")
baseline_model.fit(Tweets_train, Labels_train)


baseline_predictions = baseline_model.predict(Tweets_test)
baseline_accuracy = accuracy_score(Labels_test, baseline_predictions)

print("Most Frequent Baseline Accuracy: ", baseline_accuracy)

Most Frequent Baseline Accuracy:  0.427


# Part 3

In [33]:
model_reg = LogisticRegression(max_iter=200)

model_reg.fit(Tweets_train, Labels_train)
reg_predictions = model_reg.predict(Tweets_test)
reg_accuracy = accuracy_score(Labels_test, reg_predictions)
print("Logistic Regression Accuracy:",reg_accuracy)

#Model is slightly better than Naive Bayes

Logistic Regression Accuracy: 0.587


In [39]:
model_reg.coef_.shape

(3, 15122)

In [37]:

features = Tweets_train.shape[1]
#(bias+features) * class
total_parameters = (features + 1) * 3

print("Number of features:", features)

print("parameters in the model: ", total_parameters)

Number of features: 15122
parameters in the model:  45369


# Part 4


# Task
The task is to improve the sentiment analysis model for the `Tweets_5K.csv` dataset by:
1. Pre-processing the tweet text to remove URLs, user mentions, and hashtags.
2. Replacing the existing Bag-of-Words vectorization with TF-IDF vectorization.
3. Retraining the Logistic Regression model with the new TF-IDF features and evaluating its performance.
4. Comparing the new model's accuracy against previous models (Naive Bayes, baseline, and the initial Logistic Regression model) and explaining the results.
5. Analyzing misclassified tweets from the best-performing model to identify error patterns and providing examples.
6. Suggesting further improvements based on the misclassification analysis.
7. Finally, summarize the pre-processing, feature engineering, model performance, misclassification insights, and future recommendations.

## Pre-process Tweets (Remove URLs, Mentions, Hashtags)

### Subtask:
Clean the raw tweet text by removing URLs, user mentions (e.g., @username), and hashtags (e.g., #topic) using regular expressions. This will create a cleaner text input for vectorization.


**Reasoning**:
To clean the tweet text, I need to import the `re` module for regular expressions and define a function to remove URLs, mentions, and hashtags. Then, I will apply this function to the `raw_tweets` Series.



In [47]:
#I used URls, hashtags and mentions because on high level I see too much of these in dataset, removing them will help creating valuable vocab
def process_tweet(tweet):
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE)
    tweet = re.sub(r'@\w+', '', tweet)
    tweet = re.sub(r'#\w+', '', tweet)
    return tweet

cleaned_tweets = raw_tweets.apply(process_tweet)


print('Original tweet example where cleaning might apply:')
print(raw_tweets.loc[5]) # This tweet contains a URL
print('Cleaned tweet example:')
print(cleaned_tweets.loc[5])

Original tweet example where cleaning might apply:
http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth
Cleaned tweet example:
 - some shameless plugging for the best Rangers forum on earth


In [49]:
Tweets_train_tfidf, Tweets_test_tfidf, Labels_train_tfidf, Labels_test_tfidf = train_test_split(tfidf_df, labels, test_size=0.2, shuffle=False)

model_reg_tfidf = LogisticRegression(max_iter=200)
model_reg_tfidf.fit(Tweets_train_tfidf, Labels_train_tfidf)

reg_predictions_tfidf = model_reg_tfidf.predict(Tweets_test_tfidf)
reg_accuracy_tfidf = accuracy_score(Labels_test_tfidf, reg_predictions_tfidf)


print("Logistic Regression Accuracy with TF-IDF:", reg_accuracy_tfidf)


Logistic Regression Accuracy with TF-IDF: 0.611


In [55]:

misclassified_indices = np.where(reg_predictions_tfidf != Labels_test_tfidf)[0]

print("wrongly predicted tweets by Logistic Regression (TF-IDF): ",len(misclassified_indices))


print("Examples of misclassified tweets:\n")
num_examples = min(10, len(misclassified_indices))

for i in range(num_examples):
    idx = misclassified_indices[i]

    original_df_index = Tweets_test_tfidf.index[idx]

    print("Tweet: :",cleaned_tweets.loc[original_df_index])
    print("True Sentiment: :",Labels_test_tfidf[idx])
    print("Predicted Sentiment: :",reg_predictions_tfidf[idx])
    print("\n")


wrongly predicted tweets by Logistic Regression (TF-IDF):  389
Examples of misclassified tweets:

Tweet: : Glad I went out, glad I didn`t leave early, and glad to be afterpartying it up @ Beth`s  I`m back!
True Sentiment: : 1
Predicted Sentiment: : 0


Tweet: : is texting
True Sentiment: : 0
Predicted Sentiment: : -1


Tweet: : on my way to school of my last friday of high school ever!  and i don`t even get to see holly Gabbie and hannah
True Sentiment: : -1
Predicted Sentiment: : 0


Tweet: :  i cant get through
True Sentiment: : -1
Predicted Sentiment: : 0


Tweet: : no clubs...no parties...ive spent my friday and saturday night workin on music and am quit content
True Sentiment: : 0
Predicted Sentiment: : -1


Tweet: : Whaaaat a strong rain just came over us here in Santa Clara. I wish I could sleep but I got to attend to an important meeting
True Sentiment: : -1
Predicted Sentiment: : 0


Tweet: : Won a gps at post prom
True Sentiment: : 1
Predicted Sentiment: : 0


Tweet: : Watchi

In [56]:
#DOmain specific lexicons should be used because most of the social media texts are short form(em, ngl, op), consists of lot of emojis, different linguistic pattern

DOmain specific lexicons should be used because most of the social media texts are short form(em, ngl, op), consists of lot of emojis,  different linguistic pattern sometimes it's difficult to even read and hardly makes sense

General-purpose TF-IDF might miss the specific nuances and evolving vocabulary used in tweets, where words can have different sentiment connotations than in formal text.